In [1]:
# Parameters
municipio = "Natal"


In [2]:
# Parameters
municipio = "Natal"


Para rodar, é necessário:
- Criar pastas com os seguintes nomes: dados, img e pdfs
- Colocar os dados necessários na pasta 'dados' (caged.csv, caged_nordeste.csv, cagedest_rn_saldo_mensal.csv, porte_mensal_rn_excel.xlsx e cnaes_subclasses.csv)
    - Obs.: rodar 'build_caged.R' para obter esses dados.
- Instalar as bibliotecas que estão comentadas abaixo

In [3]:
# !pip install weasyprint
# !pip install geobr
# !pip install adjustText

In [4]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import base64
from io import BytesIO
import weasyprint
from pathlib import Path
import matplotlib.dates as mdates
import geopandas as gpd
from shapely.geometry import box
import geobr
import matplotlib.pyplot as plt
from adjustText import adjust_text
from datetime import datetime
from pathlib import Path

# Dados

In [5]:
data = pd.read_csv("dados/caged.csv")

In [6]:
data.head()

,competênciamov,região,uf,município,seção,subclasse,saldomovimentação,cbo2002ocupação,categoria,graudeinstrução,...,unidadesaláriocódigo,valorsaláriofixo,competênciaexc,indicadordeexclusão,admitidos,desligados,NI,grande_grupamento,porte,data
0,202001,2,24,240810,G,4511102,-1,521110,101,6,...,5,103900.0,NaN,NaN,0,-1,NaN,Comércio,Micro,2020-01-01
1,202001,2,24,240810,Q,8630502,1,514320,101,7,...,5,106078.0,NaN,NaN,1,0,NaN,Serviços,Média,2020-01-01
2,202001,2,24,240810,N,8111700,-1,517410,101,3,...,5,96500.0,NaN,NaN,0,-1,NaN,Serviços,Grande,2020-01-01
3,202001,2,24,241420,I,5510801,-1,422105,999,7,...,99,NaN,NaN,NaN,0,-1,NaN,Serviços,Micro,2020-01-01
4,202001,2,24,240800,N,8299799,1,524105,101,7,...,5,103900.0,NaN,NaN,1,0,NaN,Serviços,Pequena,2020-01-01


In [7]:
rn_map = geobr.read_municipality(code_muni="RN")
rn_map['code_muni'] = rn_map['code_muni'].astype(str).str[:-3]
data['município'] = data['município'].astype(str)

In [8]:
data = data.merge(
    rn_map[['code_muni', 'name_muni']],
    left_on='município',
    right_on='code_muni',
    how='left'
)


In [9]:
data['name_muni'].unique()

array(['Natal', 'Tibau Do Sul', 'Mossoró', 'Pendências', 'Parnamirim',
       'Açu', 'Extremoz', 'São José De Mipibu',
       'Governador Dix-Sept Rosado', 'Macaíba', 'Parelhas',
       'São Gonçalo Do Amarante', 'Passa E Fica', 'Santa Maria',
       'Caraúbas', 'São Miguel', 'Cruzeta', 'Pau Dos Ferros', 'Caicó',
       'Ipanguaçu', 'Apodi', 'São Bento Do Norte', 'Jardim Do Seridó',
       'Nísia Floresta', 'Baraúna', 'São Paulo Do Potengi', 'Arês',
       'Goianinha', 'Canguaretama', 'Currais Novos', 'Nova Cruz',
       'Ceará-Mirim', 'Carnaúba Dos Dantas', 'Santa Cruz',
       'São José Do Seridó', 'Espírito Santo', 'Santo Antônio',
       'Jardim De Piranhas', 'João Câmara', 'Areia Branca', 'Carnaubais',
       'Equador', 'Afonso Bezerra', 'Jundiá', 'Alto Do Rodrigues',
       'Janduís', 'Florânia', 'Vila Flor', 'Jandaíra', 'Guamaré',
       'Ouro Branco', 'Touros', 'Tangará', 'Lagoa Nova', 'Sítio Novo',
       'Venha-Ver', 'Angicos', 'São Miguel Do Gostoso', 'Parazinho',
       'Ma

In [10]:
# minicipio="Natal"

In [11]:
municipio_nome_limpo = municipio.replace(" ", "_").replace("'", "")

In [12]:
data = data[data["name_muni"]==municipio]

In [13]:
subclasses = pd.read_csv("dados/cnaes_subclasses.csv",encoding='latin1', sep=';')

In [14]:
subclasses['num_cnae'] = subclasses['num_cnae'].fillna(0).astype(int)

In [15]:
subclasses.head()

,CNAE.CNAE_SUBCLASSE,num_cnae,desc_cnae
0,"7722500 - Aluguel de fitas de vídeo, DVDs e si...",7722500,"Aluguel de fitas de vídeo, DVDs e similares"
1,8630503 - Atividade médica ambulatorial restri...,8630503,Atividade médica ambulatorial restrita a consu...
2,133402 - Cultivo de banana,133402,Cultivo de banana
3,1321900 - Tecelagem de fios de algodão,1321900,Tecelagem de fios de algodão
4,8413200 - Regulação das atividades econômicas,8413200,Regulação das atividades econômicas


In [16]:
# Realizar o merge com base na correspondência entre 'num_cnae' e 'subclasse'
data = data.merge(subclasses[['num_cnae', 'desc_cnae']], 
                  how='left', 
                  left_on='subclasse', 
                  right_on='num_cnae')

In [17]:
data.head()

,competênciamov,região,uf,município,seção,subclasse,saldomovimentação,cbo2002ocupação,categoria,graudeinstrução,...,admitidos,desligados,NI,grande_grupamento,porte,data,code_muni,name_muni,num_cnae,desc_cnae
0,202001,2,24,240810,G,4511102,-1,521110,101,6,...,0,-1,NaN,Comércio,Micro,2020-01-01,240810,Natal,4511102.0,"Comércio a varejo de automóveis, camionetas e ..."
1,202001,2,24,240810,Q,8630502,1,514320,101,7,...,1,0,NaN,Serviços,Média,2020-01-01,240810,Natal,8630502.0,Atividade médica ambulatorial com recursos par...
2,202001,2,24,240810,N,8111700,-1,517410,101,3,...,0,-1,NaN,Serviços,Grande,2020-01-01,240810,Natal,8111700.0,"Serviços combinados para apoio a edifícios, ex..."
3,202001,2,24,240810,G,4781400,-1,261125,999,9,...,0,-1,NaN,Comércio,Micro,2020-01-01,240810,Natal,4781400.0,Comércio varejista de artigos do vestuário e a...
4,202001,2,24,240810,H,5310502,1,519110,101,7,...,1,0,NaN,Serviços,Micro,2020-01-01,240810,Natal,5310502.0,Atividades de franqueadas e permissionárias do...


In [18]:
data.shape

(882954, 40)

In [19]:
data['ano'] = data['competênciamov'].astype(str).str[:4]

In [20]:
data['data'] = pd.to_datetime(data['data'])
# Encontrar a data mais recente
data_mais_recente = data['data'].max()

# Converter para o formato 'YYYYMM' como string
competencia = data_mais_recente.strftime('%Y%m')

# Exibir a competência
print("Competência:", competencia)

Competência: 202505


In [21]:
import locale

# Configurar o locale para o idioma português do Brasil
locale.setlocale(locale.LC_TIME, 'pt_BR.UTF-8')

# Converter a data mais recente para um texto por extenso
mes_referente = data_mais_recente.strftime('%B de %Y').capitalize()

# Exibir a data por extenso
print("Data por extenso:", mes_referente)


Data por extenso: Maio de 2025


In [22]:
meses_pt = {
    1: 'janeiro', 2: 'fevereiro', 3: 'março',
    4: 'abril', 5: 'maio', 6: 'junho',
    7: 'julho', 8: 'agosto', 9: 'setembro',
    10: 'outubro', 11: 'novembro', 12: 'dezembro'
}

data_mais_recente = pd.to_datetime(data['data']).max()
mes_referente = f"{meses_pt[data_mais_recente.month]} de {data_mais_recente.year}"
mes_referente = mes_referente.capitalize()


In [23]:
print("Data por extenso:", mes_referente)

Data por extenso: Maio de 2025


In [24]:
mes_nome = mes_referente.split()[0]
ano = mes_referente.split()[-1]

In [25]:
data.columns

Index(['competênciamov', 'região', 'uf', 'município', 'seção', 'subclasse',
       'saldomovimentação', 'cbo2002ocupação', 'categoria', 'graudeinstrução',
       'idade', 'horascontratuais', 'raçacor', 'sexo', 'tipoempregador',
       'tipoestabelecimento', 'tipomovimentação', 'tipodedeficiência',
       'indtrabintermitente', 'indtrabparcial', 'salário', 'tamestabjan',
       'indicadoraprendiz', 'origemdainformação', 'competênciadec',
       'indicadordeforadoprazo', 'unidadesaláriocódigo', 'valorsaláriofixo',
       'competênciaexc', 'indicadordeexclusão', 'admitidos', 'desligados',
       'NI', 'grande_grupamento', 'porte', 'data', 'code_muni', 'name_muni',
       'num_cnae', 'desc_cnae', 'ano'],
      dtype='object')

In [26]:
data['competênciamov'] = data['competênciamov'].astype(str)

admissoes = data.loc[(data['saldomovimentação'] == 1) & (data['competênciamov'] == competencia)]
total_admissoes = admissoes['saldomovimentação'].sum()

desligamentos = data.loc[(data['saldomovimentação'] == -1) & (data['competênciamov'] == competencia)]
total_desligamentos = -desligamentos['saldomovimentação'].sum()

saldo = data.loc[(data['competênciamov'] == competencia)]['saldomovimentação'].sum()

In [27]:
data['data'] = pd.to_datetime(data['data'])

In [28]:
data.head()

,competênciamov,região,uf,município,seção,subclasse,saldomovimentação,cbo2002ocupação,categoria,graudeinstrução,...,desligados,NI,grande_grupamento,porte,data,code_muni,name_muni,num_cnae,desc_cnae,ano
0,202001,2,24,240810,G,4511102,-1,521110,101,6,...,-1,NaN,Comércio,Micro,2020-01-01,240810,Natal,4511102.0,"Comércio a varejo de automóveis, camionetas e ...",2020
1,202001,2,24,240810,Q,8630502,1,514320,101,7,...,0,NaN,Serviços,Média,2020-01-01,240810,Natal,8630502.0,Atividade médica ambulatorial com recursos par...,2020
2,202001,2,24,240810,N,8111700,-1,517410,101,3,...,-1,NaN,Serviços,Grande,2020-01-01,240810,Natal,8111700.0,"Serviços combinados para apoio a edifícios, ex...",2020
3,202001,2,24,240810,G,4781400,-1,261125,999,9,...,-1,NaN,Comércio,Micro,2020-01-01,240810,Natal,4781400.0,Comércio varejista de artigos do vestuário e a...,2020
4,202001,2,24,240810,H,5310502,1,519110,101,7,...,0,NaN,Serviços,Micro,2020-01-01,240810,Natal,5310502.0,Atividades de franqueadas e permissionárias do...,2020


In [29]:
cagedest_rn_saldo_mensal =  pd.read_csv("dados/cagedest_rn_saldo_mensal.csv")
print(cagedest_rn_saldo_mensal)

           data  Saldo
0    2007-01-01    351
1    2007-02-01  -1394
2    2007-03-01  -1076
3    2007-04-01   1615
4    2007-05-01   1355
..          ...    ...
151  2019-08-01   3951
152  2019-09-01   2513
153  2019-10-01   3025
154  2019-11-01   1597
155  2019-12-01  -3133

[156 rows x 2 columns]


In [30]:
porte_mensal_rn_excel =  pd.read_excel("dados/porte_mensal_rn_excel.xlsx")
print(porte_mensal_rn_excel)

          data    porte  Saldo
0   2020-01-01   Grande  -1692
1   2020-01-01    Micro   1685
2   2020-01-01    Média   -870
3   2020-01-01  Pequena   -197
4   2020-02-01   Grande  -3293
..         ...      ...    ...
255 2025-04-01  Pequena    -58
256 2025-05-01   Grande    -92
257 2025-05-01    Micro   1814
258 2025-05-01    Média    232
259 2025-05-01  Pequena    266

[260 rows x 3 columns]


In [31]:
caged_nordeste =  pd.read_csv("dados/caged_nordeste.csv")
print(caged_nordeste)

   uf  Saldo               Estado
0  21   3560             Maranhão
1  22   3559                Piauí
2  23   5769                Ceará
3  24   2220  Rio Grande do Norte
4  25   5905              Paraíba
5  26   9754           Pernambuco
6  27    324              Alagoas
7  28   1939              Sergipe
8  29  12858                Bahia


In [32]:
# Dicionário de estoques por município (dezembro de 2023)
estoque_por_municipio = {
    "Natal": 221245,
    "Mossoró": 70587,
    "Parnamirim": 45464,
    "São Gonçalo do Amarante": 13185,
    "Macaíba": 11349,
    "Caicó": 9769,
    "Açu": 7094,
    "São José de Mipibu": 6083,
    "Ceará-Mirim": 5022,
    "Currais Novos": 4961,
    "Pau dos Ferros": 4171,
    "Tibau do Sul": 3851,
    "Baraúna": 3555,
    "Macau": 3508,
    "Arês": 3296,
    "Parelhas": 3263,
    "João Câmara": 3233,
    "Apodi": 3230,
    "Extremoz": 3065,
    "Areia Branca": 2937,
    "Baía Formosa": 2883,
    "Santa Cruz": 2672,
    "Goianinha": 2585,
    "Nova Cruz": 2512,
    "Alto do Rodrigues": 2498,
    "Nísia Floresta": 2417,
    "Canguaretama": 2163,
    "Touros": 2131,
    "Jucurutu": 2040,
    "Caraúbas": 1657,
    "Pendências": 1448,
    "Guamaré": 1392,
    "Grossos": 1316,
    "São Paulo do Potengi": 1250,
    "Ipanguaçu": 1237,
    "Jardim do Seridó": 1227,
    "Lajes": 1141,
    "Monte Alegre": 1109,
    "Carnaúba dos Dantas": 1083,
    "Itajá": 1049,
    "Santo Antônio": 1049,
    "São Miguel do Gostoso": 1033,
    "Acari": 1028,
    "Serra Negra do Norte": 1012,
    "Jardim de Piranhas": 950,
    "Espírito Santo": 930,
    "Governador Dix-Sept Rosado": 883,
    "Lagoa Nova": 863,
    "São Miguel": 837,
    "São José do Seridó": 804,
    "Jandaíra": 749,
    "Angicos": 739,
    "Tangará": 721,
    "Serra do Mel": 693,
    "Cruzeta": 680,
    "Tibau": 672,
    "Upanema": 667,
    "Porto do Mangue": 611,
    "Serra Caiada": 601,
    "Pedro Velho": 574,
    "Martins": 571,
    "Carnaubais": 567,
    "Afonso Bezerra": 544,
    "São Bento do Norte": 539,
    "Brejinho": 487,
    "Maxaranguape": 480,
    "Caiçara do Norte": 479,
    "Cerro Corá": 455,
    "Alexandria": 452,
    "Florânia": 432,
    "Ielmo Marinho": 413,
    "Umarizal": 408,
    "Janduís": 403,
    "Passa e Fica": 403,
    "Vera Cruz": 403,
    "Patu": 402,
    "São Vicente": 377,
    "Barcelona": 367,
    "Taipu": 365,
    "Equador": 364,
    "Rio do Fogo": 357,
    "Riachuelo": 352,
    "Galinhos": 329,
    "Montanhas": 329,
    "Bento Fernandes": 328,
    "Poço Branco": 309,
    "Pureza": 282,
    "São Fernando": 280,
    "Tenente Ananias": 276,
    "Santana do Matos": 275,
    "Vila Flor": 266,
    "São Tomé": 260,
    "Serra de São Bento": 260,
    "Pedro Avelino": 252,
    "Januário Cicco": 238,
    "Santa Maria": 236,
    "Olho d'Água do Borges": 228,
    "Senador Elói de Souza": 226,
    "São Francisco do Oeste": 224,
    "Bom Jesus": 222,
    "Santana do Seridó": 221,
    "Severiano Melo": 208,
    "Pedra Preta": 204,
    "Messias Targino": 201,
    "Lagoa de Velhos": 200,
    "Ouro Branco": 195,
    "Tenente Laurentino Cruz": 187,
    "João Dias": 186,
    "Monte das Gameleiras": 178,
    "São José do Campestre": 178,
    "Lagoa d'Anta": 162,
    "Lagoa Salgada": 155,
    "Riacho de Santana": 155,
    "São Pedro": 155,
    "Felipe Guerra": 154,
    "Encanto": 149,
    "Itaú": 148,
    "Francisco Dantas": 144,
    "Parazinho": 126,
    "Lucrécia": 125,
    "Campo Grande": 120,
    "Marcelino Vieira": 119,
    "Caiçara do Rio do Vento": 116,
    "Lagoa de Pedras": 116,
    "José da Penha": 113,
    "Campo Redondo": 111,
    "Portalegre": 110,
    "Fernando Pedroza": 108,
    "São Rafael": 108,
    "Serrinha dos Pintos": 106,
    "Riacho da Cruz": 104,
    "Várzea": 100,
    "Antônio Martins": 99,
    "Jaçanã": 96,
    "Triunfo Potiguar": 96,
    "São João do Sabugi": 95,
    "Almino Afonso": 94,
    "Senador Georgino Avelino": 93,
    "Timbaúba dos Batistas": 91,
    "Pedra Grande": 87,
    "Japi": 86,
    "Luís Gomes": 86,
    "Doutor Severiano": 80,
    "Bodó": 73,
    "Rafael Fernandes": 72,
    "Paraú": 70,
    "Serrinha": 61,
    "Passagem": 59,
    "Rafael Godeiro": 59,
    "Rodolfo Fernandes": 56,
    "São Bento do Trairí": 53,
    "Lajes Pintadas": 50,
    "Água Nova": 49,
    "Frutuoso Gomes": 45,
    "Pilões": 41,
    "Coronel Ezequiel": 40,
    "Ipueira": 30,
    "Sítio Novo": 30,
    "Taboleiro Grande": 30,
    "Coronel João Pessoa": 27,
    "Major Sales": 27,
    "Jundiá": 18,
    "Paraná": 16,
    "Viçosa": 14,
    "Ruy Barbosa": 12,
    "Jardim de Angicos": 8,
    "Venha-Ver": 2
}

estoque_referencia_dez_2023 = estoque_por_municipio.get(municipio, 0)
print(f"Estoque de {municipio} em dez/2023: {estoque_referencia_dez_2023}")


Estoque de Natal em dez/2023: 221245


In [33]:
# Filtrar e calcular o saldo a partir de 2024-01-01
saldo_total = porte_mensal_rn_excel[porte_mensal_rn_excel['data'] >= "2024-01-01"]['Saldo'].sum()

# Calcular o estoque final
estoque = estoque_referencia_dez_2023 + saldo_total

# Exibir o resultado
print("Estoque:", estoque)

Estoque: 260753


In [34]:
# Ajustando formato dos números
total_admissoes = f"{total_admissoes:,}".replace(",", ".")
saldo = f"{saldo:,}".replace(",", ".")
total_desligamentos = f"{total_desligamentos:,}".replace(",", ".")
estoque = f"{estoque:,}".replace(",", ".")

In [35]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import calendar

def criar_grafico_acumulado_ano_e_mes_mais_recente(data):
    # Garantir que a coluna 'data' esteja no formato datetime
    data['data'] = pd.to_datetime(data['data'], errors='coerce')

    # Criar uma nova coluna 'ano' e 'mes' para agrupamento
    data['ano'] = data['data'].dt.year
    data['mes'] = data['data'].dt.month

    # Identificar o mês mais recente da base
    mes_mais_recente = data['data'].max().month
    nome_mes_mais_recente = calendar.month_name[mes_mais_recente]  # Nome do mês (ex: Novembro)

    # Filtrar os últimos 4 anos
    ano_atual = data['ano'].max()
    anos_filtrados = list(range(ano_atual - 4, ano_atual + 1))  # Últimos 4 anos
    data_filtrada = data[data['ano'].isin(anos_filtrados)]

    # Calcular o saldo acumulado do ano até o mês mais recente
    saldo_acumulado = data_filtrada[data_filtrada['mes'] <= mes_mais_recente].groupby('ano')['saldomovimentação'].sum().reset_index()
    saldo_acumulado.rename(columns={'saldomovimentação': 'Saldo acumulado'}, inplace=True)


    # Calcular o saldo do mês mais recente
    saldo_mes = data_filtrada[data_filtrada['mes'] == mes_mais_recente].groupby('ano')['saldomovimentação'].sum().reset_index()
    saldo_mes.rename(columns={'saldomovimentação': f'Saldo {nome_mes_mais_recente}'}, inplace=True)

    # Unir os dois conjuntos de dados
    merged_data = pd.merge(saldo_acumulado, saldo_mes, on='ano', how='outer').fillna(0)

    # Transformar os dados para formato longo (long format) para o gráfico
    melted_data = pd.melt(merged_data, id_vars='ano', value_vars=['Saldo acumulado', f'Saldo {nome_mes_mais_recente}'],
                          var_name='Tipo', value_name='Saldo')

    # Criar o gráfico de barras
    plt.figure(figsize=(12, 6))
    ax = sns.barplot(x='ano', y='Saldo', hue='Tipo', data=melted_data, palette='Blues_d')

    # Remover o título da legenda
    ax.legend(title=None)

    # Títulos e rótulos
    plt.xlabel("Ano", fontsize=12)
    plt.ylabel("Saldo", fontsize=12)

    # Adicionar valores no topo de cada barra com a cor da barra
    for p in ax.patches:
        height = p.get_height()
        color = p.get_facecolor()  # Obtém a cor da barra
        if height != 0:  # Evitar exibir "0"
            ax.text(
                p.get_x() + p.get_width() / 2.,  # Posição horizontal
                height,  # Posição vertical (no topo da barra)
                '{:,.0f}'.format(height).replace(',', '.'),  # Formatação do número
                ha='center', va='bottom' if height >= 0 else 'top', fontsize=10, fontweight='bold',
                color=color  # Define a cor do texto igual à cor da barra
            )

    # Adicionar informações de fonte e elaboração
    plt.figtext(0.05, -0.01, 'Fonte: Novo Caged (MTE).',
                ha='left', fontsize=8, color='gray', weight='normal')
    plt.figtext(0.05, -0.03, 'Elaboração: SEBRAE/RN.',
                ha='left', fontsize=8, color='gray', weight='normal')

    # Ajustar layout
    plt.tight_layout()

    # Salvar o gráfico
    grafico_path = f'img/grafico_acumulado_ano_e_mes.png'
    plt.savefig(grafico_path, format='png', bbox_inches='tight', transparent=True)
    plt.show()

    return grafico_path

# Exemplo de chamada da função
# Suponha que `data` seja um DataFrame previamente carregado
grafico_path = criar_grafico_acumulado_ano_e_mes_mais_recente(data)
print(f"Gráfico salvo em: {grafico_path}")


Gráfico salvo em: img/grafico_acumulado_ano_e_mes.png


C:\Users\lhayane.macedo\AppData\Local\Temp\ipykernel_17560\1216288997.py:75: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [36]:
data['data'] = pd.to_datetime(data['data'], errors='coerce')

data['ano'] = data['data'].dt.year
data['mes'] = data['data'].dt.month

    # Identificar o mês mais recente da base
mes_mais_recente = data['data'].max().month
nome_mes_mais_recente = calendar.month_name[mes_mais_recente]  # Nome do mês (ex: Novembro)

    # Filtrar os últimos 4 anos
ano_atual = data['ano'].max()
anos_filtrados = list(range(ano_atual - 4, ano_atual + 1))  # Últimos 4 anos
data_filtrada = data[data['ano'].isin(anos_filtrados)]

    # Calcular o saldo acumulado do ano
saldo_acumulado = data_filtrada.groupby('ano')['saldomovimentação'].sum().reset_index()
saldo_acumulado.rename(columns={'saldomovimentação': 'Saldo acumulado'}, inplace=True)

In [37]:
# Agrupar os dados por ano e mês
saldo_por_mes = data_filtrada.groupby(['ano', 'mes'])['saldomovimentação'].sum().reset_index()

# Adicionar o nome do mês para facilitar a leitura
saldo_por_mes['nome_mes'] = saldo_por_mes['mes'].apply(lambda x: calendar.month_name[x])

# Ordenar os dados por ano e mês
saldo_por_mes = saldo_por_mes.sort_values(by=['ano', 'mes']).reset_index(drop=True)

# Exibir o resultado
print(saldo_por_mes)


     ano  mes  saldomovimentação   nome_mes
0   2021    1               1231    janeiro
1   2021    2               1929  fevereiro
2   2021    3                542     marÃ§o
3   2021    4              -1513      abril
4   2021    5                 85       maio
5   2021    6               1383      junho
6   2021    7               1665      julho
7   2021    8               1907     agosto
8   2021    9               1944   setembro
9   2021   10                917    outubro
10  2021   11               1740   novembro
11  2021   12                 -8   dezembro
12  2022    1               -142    janeiro
13  2022    2                811  fevereiro
14  2022    3                -48     marÃ§o
15  2022    4                818      abril
16  2022    5                942       maio
17  2022    6                982      junho
18  2022    7                871      julho
19  2022    8                980     agosto
20  2022    9                865   setembro
21  2022   10                441

In [38]:
soma_2024 = saldo_por_mes[saldo_por_mes['ano'] == 2024]['saldomovimentação'].sum()

# Exibir o resultado
print(f"Soma total do saldo de 2024: {soma_2024}")

Soma total do saldo de 2024: 12236


In [39]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import pandas as pd

def criar_grafico_ts(data, municipio):
    # Certifique-se de que a coluna 'data' é do tipo datetime
    if not pd.api.types.is_datetime64_any_dtype(data['data']):
        data['data'] = pd.to_datetime(data['data'])

    # Agrupar os dados por data
    grouped_data = data.groupby('data')['saldomovimentação'].sum().reset_index()

    # Calcular valor máximo absoluto para base de deslocamento proporcional
    total = grouped_data['saldomovimentação'].abs().max()

    # Criar gráfico
    plt.figure(figsize=(12, 6))
    sns.lineplot(x='data', y='saldomovimentação', data=grouped_data)
    plt.title("")
    plt.xlabel("")
    plt.ylabel("")

    # Configurar eixo X para datas
    ax = plt.gca()
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%m/%Y'))
    ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
    plt.xticks(rotation=45, fontsize=8)
    plt.tight_layout()
    ax.yaxis.grid(True, color='lightgray', linestyle='--', linewidth=0.8)

    # Adicionar valores nos pontos com deslocamento proporcional
    last_y = None
    for x, y in zip(grouped_data['data'], grouped_data['saldomovimentação']):
        color = '#134b5f' if y >= 0 else '#be4d25'
        offset = 0.02 * total if y >= 0 else -0.05 * total

        # Ajuste dinâmico para evitar sobreposição com base no deslocamento anterior
        if last_y is not None and abs(y - last_y) < 0.02 * total:
            offset = -0.02 * total if y > last_y else 0.02 * total

        plt.text(x, y + offset, '{:,.0f}'.format(y).replace(',', '.'),
                 ha='center', va='bottom', fontsize=8, color=color)

        last_y = y + offset

    # Rodapé
    plt.figtext(0.78, -0.01, 'Fonte: Novo Caged (MTE).',
                ha='left', fontsize=8, color='gray', weight='normal')
    plt.figtext(0.78, -0.03, 'Elaboração: SEBRAE/RN.',
                ha='left', fontsize=8, color='gray', weight='normal')

    # Salvar gráfico
    grafico_path = f'img/grafico_saldo_ts_{municipio.replace(" ", "_")}.png'
    plt.savefig(grafico_path, format='png', bbox_inches='tight', transparent=True)
    plt.show()

    return grafico_path

# Exemplo de uso:
grafico_ts = criar_grafico_ts(data, municipio)


C:\Users\lhayane.macedo\AppData\Local\Temp\ipykernel_17560\3480700249.py:56: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [40]:
# Filtra pela competência desejada
filtro = data[data['ano'] == int(ano)]

# Agrupa por setor e cnae, somando os saldos
grupo_saldos = filtro.groupby(['grande_grupamento', 'desc_cnae'])['saldomovimentação'].sum().reset_index()

# Para cada setor, pega o cnae com maior saldo
top_cnaes_df = grupo_saldos.sort_values('saldomovimentação', ascending=False) \
                           .groupby('grande_grupamento') \
                           .first() \
                           .reset_index()

top_cnaes_df = top_cnaes_df.rename(columns={
    'grande_grupamento': 'Setor',
    'desc_cnae': 'CNAE',
    'saldomovimentação': 'Saldo de Movimentação'
})

In [41]:
top_cnaes_df

,Setor,CNAE,Saldo de Movimentação
0,Agropecuária,Criação de frangos para corte,26
1,Comércio,"Comércio atacadista de mercadorias em geral, c...",67
2,Construção,Construção de obras de arte especiais,446
3,Indústria,"Atividades relacionadas a esgoto, exceto a ges...",2601
4,Serviços,Atividades de monitoramento de sistemas de seg...,1107


In [42]:
# Filtra pela competência desejada
filtro = data[data['ano'] == int(ano)]

# Agrupa por setor e CNAE, somando os saldos
grupo_saldos = filtro.groupby(['grande_grupamento', 'desc_cnae'])['saldomovimentação'].sum().reset_index()

# Para cada setor, pega o CNAE com MENOR saldo
bottom_cnaes_df = grupo_saldos.sort_values('saldomovimentação', ascending=True) \
                              .groupby('grande_grupamento') \
                              .first() \
                              .reset_index()

# Renomeia as colunas para manter o padrão
bottom_cnaes_df = bottom_cnaes_df.rename(columns={
    'grande_grupamento': 'Setor',
    'desc_cnae': 'CNAE',
    'saldomovimentação': 'Saldo de Movimentação'
})


In [43]:
bottom_cnaes_df

,Setor,CNAE,Saldo de Movimentação
0,Agropecuária,Cultivo de cana-de-açúcar,-45
1,Comércio,Comércio varejista de artigos do vestuário e a...,-405
2,Construção,Manutenção de estações e redes de telecomunica...,-46
3,Indústria,Tecelagem de fios de algodão,-69
4,Serviços,Seleção e agenciamento de mão de obra,-3296


In [44]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

def criar_grafico_saldo_setor(data, competencia, top_cnaes_df, bottom_cnaes_df):
    # Extrair o ano da competência
    ano = int(str(competencia)[:4])
    data_mes = data[data['ano'] == ano]

    # Agrupar os dados por setor
    grouped_data = data_mes.groupby('grande_grupamento')['saldomovimentação'].sum().reset_index()

    # Obter CNAEs por setor de acordo com o saldo
    def obter_desc_cnae(grupo, saldo):
        if saldo > 0:
            df = top_cnaes_df[top_cnaes_df['Setor'] == grupo]
            if not df.empty:
                cnae = df.iloc[0]
                texto = f"↑ {cnae['CNAE']}: {int(cnae['Saldo de Movimentação'])}"
                return quebra_linha(texto), '#154c79'
        elif saldo < 0:
            df = bottom_cnaes_df[bottom_cnaes_df['Setor'] == grupo]
            if not df.empty:
                cnae = df.iloc[0]
                texto = f"↓ {cnae['CNAE']}: {int(cnae['Saldo de Movimentação'])}"
                return quebra_linha(texto), 'red'
        return "", 'black'

    # Quebra de linha automática
    def quebra_linha(texto):
        return '\n'.join([texto[i:i+31] for i in range(0, len(texto), 31)]) if len(texto) > 31 else texto

    # Criar colunas com os CNAEs e cores
    textos_cnae = []
    cores_cnae = []

    for i, row in grouped_data.iterrows():
        texto, cor = obter_desc_cnae(row['grande_grupamento'], row['saldomovimentação'])
        textos_cnae.append(texto)
        cores_cnae.append(cor)

    grouped_data['desc_cnae'] = textos_cnae
    grouped_data['cor_cnae'] = cores_cnae

    # Ordenar setores pelo saldo
    grouped_data = grouped_data.sort_values(by='saldomovimentação', ascending=False)

    # Ajustar altura com base no número de setores
    num_setores = len(grouped_data)
    altura_figura = max(6, num_setores * 1.2)
    plt.figure(figsize=(12, altura_figura))

    # Criar gráfico
    ax = sns.barplot(x='grande_grupamento', y='saldomovimentação', data=grouped_data, palette='Blues_d')
    plt.ylabel("")
    plt.xlabel("")
    plt.axhline(0, color='black', linewidth=1)

    # Limites do eixo Y para posicionar CNAEs abaixo do eixo X
    ylim_min, ylim_max = ax.get_ylim()
    deslocamento = (ylim_max - ylim_min) * 0.05  # 5% abaixo do eixo

    for i, p in enumerate(ax.patches):
        height = p.get_height()
        cnae_text = grouped_data.iloc[i]['desc_cnae']
        cnae_color = grouped_data.iloc[i]['cor_cnae']

        # Valor no topo da barra
        if height > 0:
            y_text = height + (0.01 * (ylim_max - ylim_min))
            va_position = "bottom"
            saldo_color = 'black'
        elif height < 0:
            y_text = height - (0.01 * (ylim_max - ylim_min))
            va_position = "top"
            saldo_color = 'red'
        else:
            y_text = 0
            va_position = "bottom"
            saldo_color = 'black'

        # Texto do valor
        ax.text(p.get_x() + p.get_width() / 2., y_text,
                '{:,.0f}'.format(height).replace(',', '.'),
                ha="center", va=va_position, fontsize=10, fontweight='bold', color=saldo_color)

        # CNAE abaixo do nome do setor
        if cnae_text:
            y_tick_base = ylim_min - deslocamento
            ax.text(p.get_x() + p.get_width() / 2., y_tick_base,
                    cnae_text,
                    ha="center", va="top", fontsize=8, color=cnae_color)

    plt.tight_layout()

    # Rodapé
    plt.figtext(0.03, -0.01, 'Fonte: Novo Caged (MTE).',
                ha='left', fontsize=8, color='gray')
    plt.figtext(0.03, -0.03, 'Elaboração: SEBRAE/RN.',
                ha='left', fontsize=8, color='gray')

    # Salvar gráfico
    grafico_path = 'img/grafico_saldo_setor_acumulado.png'
    plt.savefig(grafico_path, format='png', bbox_inches='tight', transparent=True)

    return grafico_path

# Exemplo de uso:
grafico_saldo_setor = criar_grafico_saldo_setor(data, competencia, top_cnaes_df, bottom_cnaes_df)


C:\Users\lhayane.macedo\AppData\Local\Temp\ipykernel_17560\2654082401.py:54: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  ax = sns.barplot(x='grande_grupamento', y='saldomovimentação', data=grouped_data, palette='Blues_d')


In [45]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

def criar_grafico_saldo_setor(data, competencia, top_cnaes_df, bottom_cnaes_df):
    # Filtrar dados da competência
    data_ano = data[data['competênciamov'] == competencia]

    # Agrupar por setor
    grouped_data = data_ano.groupby('grande_grupamento')['saldomovimentação'].sum().reset_index()

    # Obter texto do CNAE por setor
    def obter_desc_cnae(grupo, saldo):
        if saldo > 0:
            df = top_cnaes_df[top_cnaes_df['Setor'] == grupo]
            if not df.empty:
                cnae = df.iloc[0]
                texto = f"↑ {cnae['CNAE']}: {int(cnae['Saldo de Movimentação'])}"
                return quebra_linha(texto), '#154c79'
        elif saldo < 0:
            df = bottom_cnaes_df[bottom_cnaes_df['Setor'] == grupo]
            if not df.empty:
                cnae = df.iloc[0]
                texto = f"↓ {cnae['CNAE']}: {int(cnae['Saldo de Movimentação'])}"
                return quebra_linha(texto), 'red'
        return "", 'black'

    # Quebra linha automática
    def quebra_linha(texto):
        return '\n'.join([texto[i:i+31] for i in range(0, len(texto), 31)]) if len(texto) > 31 else texto

    # Aplicar textos e cores dos CNAEs
    textos_cnae = []
    cores_cnae = []

    for _, row in grouped_data.iterrows():
        texto, cor = obter_desc_cnae(row['grande_grupamento'], row['saldomovimentação'])
        textos_cnae.append(texto)
        cores_cnae.append(cor)

    grouped_data['desc_cnae'] = textos_cnae
    grouped_data['cor_cnae'] = cores_cnae

    # Ordenar os setores
    grouped_data = grouped_data.sort_values(by='saldomovimentação', ascending=False)

    # Ajustar tamanho da figura dinamicamente
    num_setores = len(grouped_data)
    altura_figura = max(6, num_setores * 1.2)
    plt.figure(figsize=(12, altura_figura))

    # Criar gráfico
    ax = sns.barplot(x='grande_grupamento', y='saldomovimentação', data=grouped_data, palette='Blues_d')
    plt.ylabel("")
    plt.xlabel("")
    plt.axhline(0, color='black', linewidth=1)

    # Calcular intervalo do eixo Y
    ylim_min, ylim_max = ax.get_ylim()
    deslocamento = (ylim_max - ylim_min) * 0.05  # 5% de deslocamento para baixo

    for i, p in enumerate(ax.patches):
        height = p.get_height()
        cnae_text = grouped_data.iloc[i]['desc_cnae']
        cnae_color = grouped_data.iloc[i]['cor_cnae']

        # Texto no topo da barra
        if height > 0:
            y_text = height + (0.01 * (ylim_max - ylim_min))
            va_position = "bottom"
            saldo_color = 'black'
        elif height < 0:
            y_text = height - (0.01 * (ylim_max - ylim_min))
            va_position = "top"
            saldo_color = 'red'
        else:
            y_text = 0
            va_position = "bottom"
            saldo_color = 'black'

        ax.text(
            p.get_x() + p.get_width() / 2., y_text,
            '{:,.0f}'.format(height).replace(',', '.'),
            ha="center", va=va_position, fontsize=10, fontweight='bold', color=saldo_color
        )

        # Texto do CNAE abaixo do nome do setor
        if cnae_text:
            y_tick_base = ylim_min - deslocamento
            ax.text(
                p.get_x() + p.get_width() / 2., y_tick_base,
                cnae_text,
                ha="center", va="top", fontsize=8, color=cnae_color
            )

    plt.tight_layout()

    # Rodapé
    plt.figtext(0.03, -0.01, 'Fonte: Novo Caged (MTE).',
                ha='left', fontsize=8, color='gray')
    plt.figtext(0.03, -0.03, 'Elaboração: SEBRAE/RN.',
                ha='left', fontsize=8, color='gray')

    # Salvar gráfico
    grafico_path = 'img/grafico_saldo_setor_mensal.png'
    plt.savefig(grafico_path, format='png', bbox_inches='tight', transparent=True)

    return grafico_path

# Exemplo de uso:
grafico_saldo_setor = criar_grafico_saldo_setor(data, competencia, top_cnaes_df, bottom_cnaes_df)


C:\Users\lhayane.macedo\AppData\Local\Temp\ipykernel_17560\1420026703.py:53: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  ax = sns.barplot(x='grande_grupamento', y='saldomovimentação', data=grouped_data, palette='Blues_d')


In [46]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import calendar
import os

def criar_grafico_mes_anos_anteriores(data, setor):
    data['data'] = pd.to_datetime(data['data'], errors='coerce')

    mes_mais_recente = data['data'].max().month
    nome_mes_mais_recente = calendar.month_name[mes_mais_recente]

    data_filtrada = data[(data['data'].dt.month == mes_mais_recente) & (data['data'].dt.year >= 2020)]

    data_grouped = data_filtrada.groupby(data_filtrada['data'].dt.year)['saldomovimentação'].sum().reset_index()
    data_grouped.rename(columns={'saldomovimentação': 'Saldo', 'data': 'Ano'}, inplace=True)
    data_grouped.rename(columns={data_grouped.columns[0]: 'Ano'}, inplace=True)

    plt.figure(figsize=(12, 6))
    ax = sns.barplot(x='Ano', y='Saldo', data=data_grouped, color="#1574bc")

    plt.xlabel(f"Ano ({nome_mes_mais_recente})", fontsize=12)
    plt.ylabel("Saldo de Movimentações", fontsize=12)

    for p in ax.patches:
        height = p.get_height()
        cor = '#1574bc' if height >= 0 else '#d62728'
        va = 'bottom' if height >= 0 else 'top'
        ax.text(
            p.get_x() + p.get_width() / 2.,
            height,
            '{:,.0f}'.format(height).replace(',', '.'),
            ha='center', va=va, fontsize=10, fontweight='bold', color=cor
        )

    plt.figtext(0.1, -0.01, 'Fonte: Novo Caged (MTE).', ha='left', fontsize=8, color='gray')
    plt.figtext(0.1, -0.03, 'Elaboração: SEBRAE/RN.', ha='left', fontsize=8, color='gray')

    os.makedirs("img", exist_ok=True)
    grafico_path = f'img/grafico_mes_colunas_{municipio_nome_limpo}.png'
    plt.savefig(grafico_path, format='png', bbox_inches='tight', transparent=True)
    plt.close()  # evitar múltiplos gráficos em notebook/pipeline

    return grafico_path

grafico_path = criar_grafico_mes_anos_anteriores(data, municipio)
print(f"Gráfico salvo em: {grafico_path}")

Gráfico salvo em: img/grafico_mes_colunas_Natal.png


In [47]:
def criar_grafico_saldo_porte(data):
    data['competênciamov'] = data['competênciamov'].astype(str)
    # Filtrar os dados onde o ano da 'competênciamov' é igual ao ano de 'competencia'
    ano_competencia = str(competencia)[:4]
    data_ano = data[data['competênciamov'].str[:4] == ano_competencia]
    # Agrupar os dados por 'porte' e somar o saldo de movimentação
    grouped_data = data_ano.groupby('porte')['saldomovimentação'].sum().reset_index()

    # Ordenar os dados para exibir os maiores valores primeiro, se necessário
    grouped_data = grouped_data.sort_values(by='saldomovimentação', ascending=False)

    # Criar o gráfico de barras
    plt.figure(figsize=(12, 6))
    ax = sns.barplot(x='porte', y='saldomovimentação', data=grouped_data, palette='Blues_d')

    # Títulos e rótulos
    # plt.title("Saldo de Movimentação por Setor")
    plt.ylabel("")
    plt.xlabel('')  # Remove o nome do eixo X

    # Adicionar a linha horizontal no valor 0 do eixo Y
    plt.axhline(0, color='black',linewidth=1)

    # Adicionar os valores de cada barra dentro delas com formatação
    for p in ax.patches:
        # Posição de cada barra
        height = p.get_height()
        # Pegar a cor da barra (cor de preenchimento)
        bar_color = p.get_facecolor()
        # Colocar o valor na ponta da barra
        if height >= 0:
            ax.text(p.get_x() + p.get_width() / 2., height,  # Posição para barras positivas
                    '{:,.0f}'.format(height).replace(',', '.'),
                    ha="center", va="bottom", fontsize=10,
                    fontweight='bold', color=bar_color)
        else:
            ax.text(p.get_x() + p.get_width() / 2., height,  # Posição para barras negativas
                    '{:,.0f}'.format(height).replace(',', '.'),
                    ha="center", va="top", fontsize=10,
                    fontweight='bold', color=bar_color)
            
    plt.figtext(0.05, -0.01, 'Fonte: Novo Caged (MTE).',
                    ha='left', fontsize=8, color='gray', weight='normal')
    plt.figtext(0.05, -0.03, 'Elaboração: SEBRAE/RN.',
                    ha='left', fontsize=8, color='gray', weight='normal')

    # Exibir o gráfico
    plt.tight_layout()

    # Salvar gráfico como arquivo PNG
    grafico_path = 'img/grafico_saldo_porte.png'
    plt.savefig(grafico_path, format='png', bbox_inches='tight', transparent=True)

    return grafico_path

grafico_saldo_porte = criar_grafico_saldo_porte(data)

C:\Users\lhayane.macedo\AppData\Local\Temp\ipykernel_17560\2461217741.py:14: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  ax = sns.barplot(x='porte', y='saldomovimentação', data=grouped_data, palette='Blues_d')


In [48]:
import matplotlib.pyplot as plt
import seaborn as sns

def criar_grafico_nordeste(data):
    # Ordenar os dados por saldo para exibir os maiores valores primeiro, se necessário
    grouped_data = data.sort_values(by='Saldo', ascending=False)

    # Customizar as cores das barras
    colors = ['#3a4e5c' if nome == 'Rio Grande do Norte' else '#6c9dbf' for nome in grouped_data['Estado']]

    # Criar o gráfico de barras
    plt.figure(figsize=(12, 6))
    ax = sns.barplot(x='Estado', y='Saldo', data=grouped_data, palette=colors)

    # Títulos e rótulos
    plt.ylabel("")  # Remove o nome do eixo Y
    plt.xlabel("")  # Remove o nome do eixo X

    # Adicionar uma linha horizontal no valor 0 do eixo Y
    plt.axhline(0, color='black', linewidth=1)

    # Adicionar os valores de cada barra no gráfico
    for p, color in zip(ax.patches, colors):
        # Obter a altura da barra
        height = p.get_height()
        # Adicionar rótulo na parte superior da barra (ou inferior se for negativo)
        if height >= 0:
            ax.text(p.get_x() + p.get_width() / 2., height,  # Para barras positivas
                    '{:,.0f}'.format(height).replace(',', '.'),
                    ha="center", va="bottom", fontsize=10,
                    fontweight='bold', color=color)
        else:
            ax.text(p.get_x() + p.get_width() / 2., height,  # Para barras negativas
                    '{:,.0f}'.format(height).replace(',', '.'),
                    ha="center", va="top", fontsize=10,
                    fontweight='bold', color=color)
            
    plt.figtext(0.05, -0.01, 'Fonte: Novo Caged (MTE).',
                    ha='left', fontsize=8, color='gray', weight='normal')
    plt.figtext(0.05, -0.03, 'Elaboração: SEBRAE/RN.',
                    ha='left', fontsize=8, color='gray', weight='normal')

    # Ajustar layout
    plt.tight_layout()

    # Salvar o gráfico como arquivo PNG
    grafico_path = 'img/grafico_nordeste.png'
    plt.savefig(grafico_path, format='png', bbox_inches='tight', transparent=True)

    return grafico_path

# Exemplo de chamada da função
grafico_nordeste = criar_grafico_nordeste(caged_nordeste)
print(f"Gráfico salvo em: {grafico_nordeste}")


C:\Users\lhayane.macedo\AppData\Local\Temp\ipykernel_17560\1270981976.py:13: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  ax = sns.barplot(x='Estado', y='Saldo', data=grouped_data, palette=colors)


Gráfico salvo em: img/grafico_nordeste.png


In [49]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import pandas as pd

def criar_grafico_ts_porte(data):
    # Converter a coluna 'competênciamov' para string, caso não seja
    data['competênciamov'] = data['competênciamov'].astype(str)

    # Filtrar os dados onde o ano da 'competênciamov' é igual ao ano de 'competencia'
    ano_competencia = str(competencia)[:4]
    data_ano = data[data['competênciamov'].str[:4] == ano_competencia]

    # Garantir que a coluna 'data' seja do tipo datetime
    data_ano['data'] = pd.to_datetime(data_ano['data'])

    # Agrupar os dados por 'data' e 'porte', somando os valores de 'saldomovimentação'
    grouped_data = data_ano.groupby(['data', 'porte'])['saldomovimentação'].sum().reset_index()

    # Obter a lista de valores únicos de 'porte' e associar cores
    unique_porte = grouped_data['porte'].unique()
    palette = sns.color_palette("muted", n_colors=len(unique_porte))
    color_mapping = dict(zip(unique_porte, palette))

    # Criar o gráfico com hue para a coluna 'porte'
    plt.figure(figsize=(12, 6))
    ax = sns.lineplot(x='data', y='saldomovimentação', hue='porte', data=grouped_data, 
                      marker="o", palette=color_mapping)

    # Títulos e rótulos (ajustáveis conforme necessidade)
    plt.title("")
    plt.xlabel("")
    plt.ylabel("")

    # Adicionar a linha cinza no eixo Y = 0
    plt.axhline(0, color='lightgray', linestyle='--', linewidth=1)
    
    # Formatar os rótulos do eixo X para exibir apenas mês/ano
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m/%Y'))
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=1))  # Exibe de 1 em 1 mês
    plt.xticks(rotation=45)

    # Configurar os ticks do eixo X para garantir que todos os pontos sejam exibidos
    unique_dates = grouped_data['data'].unique()
    plt.xticks(ticks=unique_dates, labels=[date.strftime('%m/%Y') for date in unique_dates])

    # Adicionar valores exatamente no centro dos pontos, com background colorido e números brancos
    for porte in grouped_data['porte'].unique():
        porte_data = grouped_data[grouped_data['porte'] == porte]
        for x, y in zip(porte_data['data'], porte_data['saldomovimentação']):
            # Adicionar texto com um fundo colorido (mesma cor da linha) e texto branco
            plt.text(
                x, y, f'{int(y):,}'.replace(',', '.'),  # Formatação do número
                ha='center', va='center', fontsize=8, color='white',  # Cor do número
                bbox=dict(facecolor=color_mapping[porte], edgecolor='none', boxstyle='round,pad=0.2')  # Fundo colorido
            )

    # Legenda dentro do gráfico no canto superior esquerdo
    plt.legend(
        title="Porte", loc="upper left", frameon=True, fontsize=8,
        title_fontsize=10, facecolor='white', edgecolor='black', fancybox=True
    )

    # Notas no rodapé
    plt.figtext(0.08, -0.01, 'Fonte: Novo Caged (MTE).',
                ha='left', fontsize=8, color='gray', weight='normal')
    plt.figtext(0.08, -0.03, 'Elaboração: SEBRAE/RN.',
                ha='left', fontsize=8, color='gray', weight='normal')

    # Salvar gráfico
    grafico_path = 'img/grafico_ts_porte.png'
    plt.savefig(grafico_path, format='png', bbox_inches='tight', transparent=True)
    plt.show()
    return grafico_path

# Uso da função
grafico_ts_porte = criar_grafico_ts_porte(data)


C:\Users\lhayane.macedo\AppData\Local\Temp\ipykernel_17560\1549733154.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_ano['data'] = pd.to_datetime(data_ano['data'])


C:\Users\lhayane.macedo\AppData\Local\Temp\ipykernel_17560\1549733154.py:73: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


# PDF

In [50]:
# Função para converter a imagem em Base64 (para embutir no HTML)
def imagem_para_base64(caminho_imagem):
    with open(caminho_imagem, "rb") as img_file:
        return base64.b64encode(img_file.read()).decode('utf-8')

# Função para gerar o conteúdo HTML
def gerar_html(admissoes, saldo, desligamentos, estoque_clt):
    grafico_acumulado_ano_e_mes ='img/grafico_acumulado_ano_e_mes.png'
    grafico_saldo_ts = f'img/grafico_saldo_ts_{municipio_nome_limpo}.png'
    grafico_saldo_setor_mensal = 'img/grafico_saldo_setor_mensal.png'
    grafico_saldo_setor_acumulado = 'img/grafico_saldo_setor_acumulado.png'
    grafico_mes_colunas = f'img/grafico_mes_colunas_{municipio_nome_limpo}.png'
    grafico_saldo_porte = 'img/grafico_saldo_porte.png'
    grafico_ts_porte = 'img/grafico_ts_porte.png'

    # Converter a imagem de cabeçalho em base64
    header_image_base64 = imagem_para_base64('img\header-emprego.png')

    # Converter o gráfico em base64
    grafico_saldo_ts_b64 = imagem_para_base64(grafico_saldo_ts)
    grafico_saldo_setor_mensal_b64 = imagem_para_base64(grafico_saldo_setor_mensal)
    grafico_saldo_setor_acumulado_b64 = imagem_para_base64(grafico_saldo_setor_acumulado)
    grafico_mes_colunas_b64 = imagem_para_base64(grafico_mes_colunas)
    grafico_saldo_porte_b64 = imagem_para_base64(grafico_saldo_porte)
    grafico_ts_porte_b64 = imagem_para_base64(grafico_ts_porte)
    grafico_acumulado_ano_e_mes_b64 = imagem_para_base64(grafico_acumulado_ano_e_mes)

    # Criar conteúdo HTML
    html_content = f"""
    <!DOCTYPE html>
    <html lang="pt-br">
    <head>
        <meta http-equiv="Content-Type" content="text/html; charset=utf-8">
        <title>Boletim de Emprego</title>
        <style>
            @page {{
                size: 1080px 720px;
                margin: 0;
            }}

            /* Definir o fundo da página */
            body {{
                background-color: #f3f3f3; /* Cor de fundo */
                font-family: Arial, sans-serif;
                margin: 0;
                padding: 0;
            }}
            header {{
                background-image: url('data:image/png;base64,{header_image_base64}');
                background-repeat: no-repeat;
                background-size: 100% 100%;
                background-position: center;
                height: 360px;
                text-align: center;
                color: white;
                font-size: 24px;
                display: flex;
                align-items: center;
                justify-content: center;
            }}
                .mes {{
                font-size: 24px;
                font-weight: bold;
                color: #333;
                text-align: center;
                margin-top: 30px;
            }}

            /* Estilo para as caixinhas */
            .informacoes {{
                display: flex;
                justify-content: space-between;
                padding: 20px;
                margin: 20px 30px;
            }}
            .caixinha {{
                background-color: #fff;
                border: 1px solid #ddd;
                padding: 30px;
                text-align: center;
                width: 22%;
                border-radius: 10px;
                box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
            }}
            .caixinha h3 {{
                font-size: 20px;
                color: #1574bc;
            }}
            .caixinha p {{
                font-size: 24px;
                font-weight: bold;
                color: #333;
            }}
            .grafico h3 {{
                font-size: 24px;
                font-weight: bold;
                color: #333;
                text-align: center;
                padding-top: 60px;
            }}

            .grafico h2 {{
                padding-top= -15px;
                font-size: 18px;
                color: #333;
                text-align: center;
                font-weight: normal;
            }}

            .grafico img {{
                width: 1000px;
                padding-top: 20px;
                padding-left: 30px;
            }}

            .tabela {{
                width: 100%;
                border-collapse: collapse;
                margin-top: 20px;
            }}
            th, td {{
                padding: 10px;
                text-align: center;
                border: 1px solid #ddd;
            }}
            th {{
                background-color: #333;
                color: white;
            }}
            td {{
                background-color: #fff;
            }}
            footer {{
                text-align: center;
                font-size: 12px;
                margin-top: 30px;
            }}
        </style>
    </head>
    <body>
        <header>
        </header>

        <!-- Seção do título do mês -->
        <div class="mes">{mes_referente} - {municipio}</div>

        <!-- Seção das Caixinhas -->
        <div class="informacoes">
            <div class="caixinha">
                <h3>Admissões</h3>
                <p>{total_admissoes}</p>
            </div>
            <div class="caixinha">
                <h3>Saldo</h3>
                <p>{saldo}</p>
            </div>
            <div class="caixinha">
                <h3>Desligamentos</h3>
                <p>{total_desligamentos}</p>
            </div>
            <div class="caixinha">
                <h3>Estoque CLT</h3>
                <p>{estoque_clt}</p>
            </div>
        </div>
        <div class="grafico">
            <h3>Saldo de Emprego Acumulado e {mes_nome} - {municipio}</h3>
            <img src="data:image/png;base64,{grafico_acumulado_ano_e_mes_b64}">
        </div>

        <div class="grafico">
            <h3>Saldo de Emprego por Mês - {municipio}</h3>
            <img src="data:image/png;base64,{grafico_saldo_ts_b64}">
        </div>

        <div class="grafico">
            <h3>Saldo de Emprego Acumulado de {ano} por Setor - {municipio}</h3>
            <img src="data:image/png;base64,{grafico_saldo_setor_acumulado_b64}">
        </div>

        <div class="grafico">
            <h3>Saldo de Emprego em {mes_nome} por Setor - {municipio}</h3>
            <img src="data:image/png;base64,{grafico_saldo_setor_mensal_b64}">
        </div>

        <div class="grafico">
            <h3>Saldo de Emprego nos Meses de {mes_nome} - {municipio}</h3>
            <img src="data:image/png;base64,{grafico_mes_colunas_b64}">
        </div>

       <div class="grafico">
            <h3>Saldo Acumulado de Emprego de {ano} por Porte - {municipio}</h3>
            <img src="data:image/png;base64,{grafico_saldo_porte_b64}">
        </div>

        <div class="grafico">
            <h3>Saldo de Emprego por Mês e Porte - {municipio}</h3>
            <img src="data:image/png;base64,{grafico_ts_porte_b64}">
        </div>

    </body>
    </html>
    """

    return html_content

def gerar_pdf(html_content, file_path):
  weasyprint.HTML(string=html_content).write_pdf(file_path)

# Gerar HTML com os valores das variáveis
html_content = gerar_html(total_admissoes, saldo, total_desligamentos, estoque)

# Gerar o PDF
gerar_pdf(html_content, f"pdfs/boletim_emprego_{competencia}-{municipio_nome_limpo}.pdf")

<>:17: SyntaxWarning: invalid escape sequence '\h'
<>:17: SyntaxWarning: invalid escape sequence '\h'
C:\Users\lhayane.macedo\AppData\Local\Temp\ipykernel_17560\2629138342.py:17: SyntaxWarning: invalid escape sequence '\h'
  header_image_base64 = imagem_para_base64('img\header-emprego.png')
